In [ ]:
function simulations(ae, sp; mean_yx = false, mean_y=false, mean_x=false) #BurnTm = 100, Tsimul = BurnTm + 250*4, Nsimul = 10
    #From ae
    @unpack newΛ, Ny, grid_yd, grid_yc, grid_c, Nc = ae #, gridpetrol, z_grid,
    @unpack index_b_c, index_b_d, grid_b, g_c, g_d, Vc, Vd = ae
    @unpack μ, q, γ, Λ, τ, newΓ, z, qs = ae
    @unpack index_bs_c, grid_bs, index_bs_d = ae
    @unpack l_c, l_d, prob_def, Nb = ae
    
    #From simulation_spaces
    @unpack BurnTm, Nsim, Tsim = sp
    @unpack SimIndexShock, SimDefault, SimBIndx, SimB,SimBsIndx, SimBs, SimLabr,SimPrice, SimIndexCop   = sp
    @unpack SimCons, SimGovt,SimDefaultEvent, SimDebt, SimIndexTaxRate,SimOutp, SimCopRev, SimCopProd = sp
    @unpack SimDbtRatio, SimGRatio, SimIntRate, SimValue, Sim_hp, SimBsRatio, Sim_escenario, SimFullOutp = sp
    @unpack SimBNRatio, SimΘ, SimΘ1, SimΘ2, SimΘ3, SimΘ4, SimBsRatiomean, SimBs_escenario, SimBs_escenario_mix, SimProb = sp
    
    #Assets
    sizeΛ = size(Λ)[2] 
    midshock = Int((sizeΛ+1)/2) 
    
    #Commodity
    sizeΓ = size(ae.Γ) 
    sizeΓ = sizeΓ[2]    
    midshockcop = (sizeΓ+1)/2     
    
    #Commodity
    indexnodebt = searchsortedfirst(grid_b, 0.)
    # println("indexnodebt :", indexnodebt)
    InitBIndx = 1   # initial B index (for conditional welfare)
    InitBsIndx = 1  # initial Bs index
    
    default_status = fill(false, (Nsim,Tsim))

    #Seed
    Random.seed!(3);
    randomshock = Array{Float64}(undef, Tsim)
    randomcop = Array{Float64}(undef, Tsim)
    randomentry = Array{Float64}(undef, Tsim) # shock to re-entry to international markets when in default            
    #Simulation of exogenous stochastic processes: TFP 
    
    
    for n = 1:Nsim

        #initial necessary variables
        SimIndexShock[n,1] = midshock # start from long run average aggregate productivity shock
        SimIndexCop[n,1]   = midshockcop # start from long run average of Cop revenues
               
        for t = 2:Tsim    
            randomshock[t] = rand(Float64)
            randomcop[t]   = rand(Float64)
            
            #aggregate productivity shock for period "t"
            if  randomshock[t] <= newΛ[SimIndexShock[n,t-1],1] 
                    SimIndexShock[n,t] = 1 
            end 
            
            for s = 1:Ny 
                
                if (randomshock[t] > newΛ[SimIndexShock[n,t-1],s]) && (randomshock[t] <= newΛ[SimIndexShock[n,t-1],s+1]) 
                    SimIndexShock[n,t] = s + 1 
                end
            end  
            
            #Cop revenue shock for period "t"
            if  randomcop[t] <= newΓ[SimIndexCop[n,t-1],1]
                    SimIndexCop[n,t] = 1 
            end   
            
            for s = 1:Nc
                if (randomcop[t] > newΓ[SimIndexCop[n,t-1],s]) && (randomcop[t] <= newΓ[SimIndexCop[n,t-1],s+1]) 
                        SimIndexCop[n,t] = s + 1 
                end 
            end    
        end 
    end  
    
    
    
    #Simulation: each "nnn" is the number of a simulation, each "ttt" the period for a specific simulation "nnn".
    for n = 1:Nsim
        
        # initial necessary variables
        SimDefault[n,1]  = 0 # economy starts w/access to credit markets (not in default)
        SimCopRev[n,2]   = z*grid_c[SimIndexCop[n,2]]  
        
        SimBIndx[n,1]  = InitBIndx
        SimBsIndx[n,1] = InitBsIndx
        
        SimBIndx[n,2]  = index_b_c[SimBIndx[n,1],  SimIndexShock[n,1], SimBsIndx[n,1], SimIndexCop[n,1]] 
        SimBsIndx[n,2] = index_bs_c[SimBIndx[n,1], SimIndexShock[n,1], SimBsIndx[n,1], SimIndexCop[n,1]] 
        
        SimB[n,1]    = grid_b[SimBIndx[n,1]] 
        SimB[n,2]    = grid_b[SimBIndx[n,2]]
        SimBs[n,1]   = grid_bs[SimBsIndx[n,1]] 
        SimBs[n,2]   = grid_bs[SimBsIndx[n,2]]
        
        SimLabr[n,1]  = l_c[SimIndexShock[n,1]] 
        # SimCons[n,1]  = c_c[SimBIndx[n,1], SimIndexShock[n,1], SimBsIndx[n,1], SimIndexCop[n,1]]        
        SimGovt[n,1]  = g_c[SimBIndx[n,1], SimIndexShock[n,1], SimBsIndx[n,1], SimIndexCop[n,1]]       
        SimValue[n,1] = Vc[SimBIndx[n,1],  SimIndexShock[n,1], SimBsIndx[n,1], SimIndexCop[n,1]]  
        
        default_status[n,1] = false
        
        for t = 2:Tsim-1
            # determine the value of cop revenues
            SimCopProd[n,t]  = z*grid_c[SimIndexCop[n,t]]
            
            #Shock productividad y cobre
            #Escenario malo
            if (grid_yc[SimIndexShock[n,t]] < 1) && (grid_c[SimIndexCop[n,t]] < 1) 
                Sim_escenario[n,t] = 0
            #Escenario bueno
            elseif (grid_yc[SimIndexShock[n,t]] > 1) && (grid_c[SimIndexCop[n,t]] > 1) 
                Sim_escenario[n,t] = 1
            #Escenarios intermedios
            else  
                Sim_escenario[n,t] = 2
            end              
            
            # previous period was not in default: default decision and the appropriate variables given the default decision:
            randomentry[t] = rand(Float64)
            tfp_nt = SimIndexShock[n,t]
            cop_nt = SimIndexCop[n,t]
            b_nt   = SimBIndx[n,t] # index_b_c[SimBIndx[n,t],SimIndexShock[n,1],SimBsIndx[n,t]] 
            bs_nt  = SimBsIndx[n,t] # index_bs_c[SimBIndx[n,t],SimIndexShock[n,1],SimBsIndx[n,t]] 
            SimProb[n,t] = prob_def[b_nt, tfp_nt, bs_nt, cop_nt]
            
            if SimDefault[n,t-1]==0  
                if Vc[b_nt, tfp_nt, bs_nt, cop_nt] > Vd[indexnodebt, tfp_nt, bs_nt, cop_nt]   

                    SimDefault[n,t]     = 0 
                    SimDefaultEvent[n,t]= 0 
                    
                    SimBIndx[n,t+1]     = index_b_c[b_nt, tfp_nt, bs_nt, cop_nt]
                    SimB[n,t+1]         = grid_b[SimBIndx[n,t+1]]
                    
                    SimBsIndx[n,t+1]    = index_bs_c[b_nt, tfp_nt, bs_nt, cop_nt]
                    SimBs[n,t+1]        = grid_bs[SimBsIndx[n,t+1]]      
                            
                    SimDebt[n,t]        = SimB[n,t+1] 
                    
                    SimLabr[n,t]        = l_c[tfp_nt]  
                    SimOutp[n,t]        = grid_yc[tfp_nt] * SimLabr[n,t]  
                    SimFullOutp[n,t]    = SimOutp[n,t] + SimCopProd[n,t] 
                    
                    SimGovt[n,t]        = g_c[b_nt, tfp_nt, bs_nt, cop_nt]     
                    SimPrice[n,t]       = q[SimBIndx[n,t+1], tfp_nt, SimBsIndx[n,t+1], cop_nt] 
                    SimDbtRatio[n,t]    = SimDebt[n,t]/(SimFullOutp[n,t]) 
                    SimBsRatio[n,t]     = SimBs[n,t]/(SimFullOutp[n,t])
                    SimGRatio[n,t]      = SimGovt[n,t]/SimFullOutp[n,t] 
                    SimIntRate[n,t]     = 1/SimPrice[n,t] - 1 
                    SimValue[n,t]       = Vc[b_nt, tfp_nt, bs_nt, cop_nt]    
                else 
                    SimDefault[n,t]     = 1 
                    SimDefaultEvent[n,t]= 1
                    
                    SimBIndx[n,t+1]       = indexnodebt 
                    SimB[n,t+1]           = grid_b[SimBIndx[n,t+1]]
                    
                    SimBsIndx[n,t+1]      = index_bs_d[b_nt, tfp_nt, bs_nt, cop_nt] #indexnodebt 
                    SimBs[n,t+1]          = grid_bs[SimBsIndx[n,t+1]]  
                    
                    SimDebt[n,t]        = NaN 
                    
                    SimLabr[n,t]        = l_d[tfp_nt]  
                    SimOutp[n,t]        = grid_yd[tfp_nt] * SimLabr[n,t]  
                    SimFullOutp[n,t]    = SimOutp[n,t] + SimCopProd[n,t] 
                    
                    SimGovt[n,t]        = g_d[tfp_nt, bs_nt, cop_nt] 
                    SimPrice[n,t]       = NaN
                    SimDbtRatio[n,t]    = NaN
                    SimBsRatio[n,t]     = SimBs[n,t]/(SimFullOutp[n,t])
                    SimGRatio[n,t]      = SimGovt[n,t]/SimFullOutp[n,t] 
                    SimIntRate[n,t]     = NaN 
                    default_status[n,t] = true
                    SimValue[n,t]       = Vd[indexnodebt, tfp_nt, bs_nt, cop_nt]  
                end 
            end  
            
        # previous period was in default: given probability of re-entry, give decision and all other variables.
            
            if SimDefault[n,t-1]==1 
                if (Vc[indexnodebt, tfp_nt, bs_nt, cop_nt] > Vd[indexnodebt, tfp_nt, bs_nt, cop_nt]) && (randomentry[t]<μ)

                    #println("C1")
                    SimDefault[n,t]     = 0  
                    SimDefaultEvent[n,t]= 0        
                    
                    SimBIndx[n,t]       = indexnodebt 
                    SimB[n,t]           = grid_b[b_nt] 
                    
                    SimBIndx[n,t+1]     = index_b_c[b_nt, tfp_nt, bs_nt, cop_nt] 
                    SimB[n,t+1]         = grid_b[SimBIndx[n,t+1]]                   
 
                    SimBsIndx[n,t]      = index_bs_c[b_nt, tfp_nt, bs_nt, cop_nt]
                    SimBs[n,t]          = grid_bs[bs_nt] 
                    
                    SimBsIndx[n,t+1]    = index_bs_c[b_nt, tfp_nt, bs_nt, cop_nt] 
                    SimBs[n,t+1]        = grid_bs[SimBsIndx[n,t+1]] 
                    
                    SimDebt[n,t]        = SimB[n,t+1]   
                    
                    SimLabr[n,t]        = l_c[tfp_nt]  
                    SimOutp[n,t]        = grid_yc[tfp_nt] * SimLabr[n,t]  
                    SimFullOutp[n,t]    = SimOutp[n,t] + SimCopProd[n,t] 
                    
                    SimGovt[n,t]        = g_c[b_nt, tfp_nt, bs_nt, cop_nt]     
                    SimPrice[n,t]       = q[SimBIndx[n,t+1], tfp_nt, SimBsIndx[n,t+1], cop_nt]         
                    SimDbtRatio[n,t]    = SimDebt[n,t]/(SimFullOutp[n,t])   
                    SimBsRatio[n,t]     = SimBs[n,t]/(SimFullOutp[n,t])
                    SimGRatio[n,t]      = SimGovt[n,t]/SimFullOutp[n,t] 
                    SimIntRate[n,t]     = 1/SimPrice[n,t] - 1        
                    SimValue[n,t]       = Vc[indexnodebt, tfp_nt, bs_nt, cop_nt]   
                else 
                    SimDefault[n,t]     = 1 
                    SimDefaultEvent[n,t]= 0       
                    
                    SimBIndx[n,t+1]       = indexnodebt 
                    SimB[n,t+1]           = grid_b[SimBIndx[n,t+1]]
                    
                    SimBsIndx[n,t+1]       = index_bs_d[b_nt, tfp_nt, bs_nt, cop_nt]  
                    SimBs[n,t+1]           = grid_bs[SimBsIndx[n,t+1]]
                                        
                    SimDebt[n,t]        = NaN    
                    
                    SimLabr[n,t]        = l_d[tfp_nt]  
                    SimOutp[n,t]        = grid_yd[tfp_nt] * SimLabr[n,t]  
                    SimFullOutp[n,t]    = SimOutp[n,t] + SimCopProd[n,t] 
                    
                    SimGovt[n,t]        = g_d[tfp_nt, bs_nt, cop_nt] 
                    SimPrice[n,t]       = NaN        
                    SimDbtRatio[n,t]    = NaN  
                    SimBsRatio[n,t]     = SimBs[n,t]/(SimFullOutp[n,t])
                    SimGRatio[n,t]      = SimGovt[n,t]/SimFullOutp[n,t]
                    SimIntRate[n,t]     = NaN # note here for log-HP we need a number       
                    default_status[n,t] = true
                    SimValue[n,t]       = Vd[indexnodebt, tfp_nt, bs_nt, cop_nt]
                    #println("D2")
                end 
            end             
        end
    end
    
    
    
    for n = 1:Nsim
        if mean_yx == false
            mean_y  =   mean(SimOutp[n,2:end-1])
            mean_x  =   mean(SimCopProd[n,2:end-1])
        end
        mean_g  =   mean(SimGovt[n,2:end-1])
        mean_bs =   mean(SimBs[n,2:end-1])
        mean_q =   mean(SimPrice[n,2:end-1][.!isnan.(SimPrice[n,2:end-1])])
        mean_bp =   mean(SimDebt[n,2:end-1][.!isnan.(SimDebt[n,2:end-1])])
        
        SimBsRatiomean[n,2:end-1] = SimBs[n,2:end-1] / (mean_y + mean_x)
        
        SimΘ[n,2:end-1]  = (τ*mean_y + mean_x .+(1 .-SimPrice[n,2:end-1]).*SimDebt[n,2:end-1] )./SimFullOutp[n,2:end-1] - (SimGovt[n,2:end-1] ./ SimFullOutp[n,2:end-1]) #  
        SimΘ1[n,2:end-1] = (τ*mean_y + mean_x .+(1 .-SimPrice[n,2:end-1]).*SimDebt[n,2:end-1] .+ (1 .-qs).*SimBs[n,2:end-1])./SimFullOutp[n,2:end-1] - (SimGovt[n,2:end-1] ./ SimFullOutp[n,2:end-1]) # 
        SimΘ2[n,2:end-1] = (τ*mean_y + mean_x .+(1 .-SimPrice[n,2:end-1]).*SimDebt[n,2:end-1] .+ (1 .-qs).*mean_bs)./SimFullOutp[n,2:end-1] - (SimGovt[n,2:end-1] ./ SimFullOutp[n,2:end-1]) # 
        SimΘ3[n,2:end-1] = (τ*mean_y + mean_x .+(1 .-mean_q).*mean_bp .+ (1 .-qs).*mean_bs)./SimFullOutp[n,2:end-1] - (SimGovt[n,2:end-1] ./ SimFullOutp[n,2:end-1]) # 
        SimΘ4[n,2:end-1] = (τ*mean_y + mean_x )./SimFullOutp[n,2:end-1] - (SimGovt[n,2:end-1] ./ SimFullOutp[n,2:end-1]) # 
                         
        SimBNRatio[n,2:end-1] = (SimDebt[n,2:end-1] .+ SimBs[n,2:end-1]) ./ SimFullOutp[n,2:end-1]
    
        
        for t = 1:Tsim
            """ Deuda alta """
            if SimDebt[n,t] <= grid_b[Int(floor(Nb*0.25)+1)] #Alta deuda
                SimBs_escenario[n,t] = 0 
                
                #Ciclo adverso
                if (grid_yc[SimIndexShock[n,t]] < 1) && (grid_c[SimIndexCop[n,t]] < 1) 
                    SimBs_escenario_mix[n,t] = 0
                #Ciclo bueno
                elseif (grid_yc[SimIndexShock[n,t]] > 1) && (grid_c[SimIndexCop[n,t]] > 1) 
                    SimBs_escenario_mix[n,t] = 1
                #Ciclo intermedio
                else  
                    SimBs_escenario_mix[n,t] = 2
                end 
                
            """ Deuda baja """
            elseif SimDebt[n,t] >= grid_b[Int(floor(Nb*0.75)+1)] #Baja deuda  
                SimBs_escenario[n,t] = 1 
                
                #Ciclo adverso
                if (grid_yc[SimIndexShock[n,t]] < 1) && (grid_c[SimIndexCop[n,t]] < 1) 
                    SimBs_escenario_mix[n,t] = 3
                #Ciclo bueno
                elseif (grid_yc[SimIndexShock[n,t]] > 1) && (grid_c[SimIndexCop[n,t]] > 1) 
                    SimBs_escenario_mix[n,t] = 4
                #Ciclo intermedio
                else  
                    SimBs_escenario_mix[n,t] = 5
                end                 
                
                
            """ Deuda intermedia """
            else
                SimBs_escenario[n,t] = 2 
                
                #Ciclo adverso
                if (grid_yc[SimIndexShock[n,t]] < 1) && (grid_c[SimIndexCop[n,t]] < 1) 
                    SimBs_escenario_mix[n,t] = 6
                #Ciclo bueno
                elseif (grid_yc[SimIndexShock[n,t]] > 1) && (grid_c[SimIndexCop[n,t]] > 1) 
                    SimBs_escenario_mix[n,t] = 7
                #Ciclo intermedio
                else  
                    SimBs_escenario_mix[n,t] = 8
                end                     
                
            end 
        end
                
    end   
    
    return SimOutp, SimIntRate, SimGovt, SimDefaultEvent, SimDefault, SimDbtRatio, SimGRatio,
           SimDebt, SimB,SimBs, default_status, SimValue, SimPrice, SimBsRatio, SimFullOutp, 
           SimCopProd, SimBNRatio, Sim_escenario, SimΘ, SimΘ1, SimΘ2, SimΘ3, SimΘ4, SimBsRatiomean, 
           SimBs_escenario, SimBs_escenario_mix, SimProb
end
